# Face ID Project for the Youth Soccor League in Dallas 

2018/7/12 Mooyoung Lee


**Reference**: 
- [OpenCV Python TUTORIAL #4 for Face Recognition and Identification](https://www.youtube.com/watch?v=PmZ29Vta7Vc)


**Objective**: Identifying person's name by reading web cam input.  

**Procedure**:  Run below code A, B, and C in order.

**Furthur direction**:  
1. This code will be changed to take only a photo instead of video.  
2. This algorithm produce low accuracy in identifying people so a deep learning algorithm need to be used. 

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import pickle


# <font color='blue'>A. Copy photos from 'pictures/Camera Roll' to 'project/data' folder

Steps:

1. Open 'Camera' app and take multiple photos of ONE person, and update 'camera_dir' path on the below
2. Update the path for your training data directory, 'data_dir'
3. Update the 'name' on the below which is the name of a person in the photos
4. Execute below section to copy photos of ONE person into a training data folder with a folder name 
5. Repeat step 3-4 for every time when you take photos of new person. 

In [25]:
# image copy

name = 'JOSHUA-lee'

camera_dir = r'C:\Users\ML\Pictures\Camera Roll'
data_dir = r'C:\Users\ML\Documents\faceID\image'
name_dir = os.path.join(data_dir, name)
# create a target directory for each person
if not os.path.exists(name_dir):
    os.makedirs(name_dir)

for root, dirs, files in os.walk(camera_dir):
    for file in files:
        if file.endswith('jpg') or file.endswith('png'):
            path_src = os.path.join(root, file)
            path_target = os.path.join(name_dir, file)
            print(path_src)
            os.rename(path_src, path_target)

C:\Users\ML\Pictures\Camera Roll\WIN_20180711_21_42_41_Pro (2).jpg
C:\Users\ML\Pictures\Camera Roll\WIN_20180711_21_42_41_Pro.jpg
C:\Users\ML\Pictures\Camera Roll\WIN_20180711_21_42_42_Pro.jpg
C:\Users\ML\Pictures\Camera Roll\WIN_20180711_21_42_43_Pro.jpg
C:\Users\ML\Pictures\Camera Roll\WIN_20180711_21_42_44_Pro.jpg
C:\Users\ML\Pictures\Camera Roll\WIN_20180711_21_42_46_Pro.jpg


# Web cam test

In [2]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Display the resulting frame
    cv2.imshow('frame',frame)
    
#     # Display grayscale frames
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     cv2.imshow('frame',gray)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

# Web cam test for a box identifying a face

In [16]:
import numpy as np
import cv2

# classifier input
face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')


# video w/rectangle
cap = cv2.VideoCapture(0)
color = (255, 0, 0) #BGR
stroke = 2


while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
   
    # Display grayscale frames
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.5, minNeighbors=5)
    for (x, y, w, h) in faces:
        
        # region of interest
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        img_item = 'my-image.png'
        cv2.imwrite(img_item, roi_gray)  # save roi image
        
        # rectable identifying faces
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x,y), (end_cord_x, end_cord_y), color, stroke)
        
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

# <font color='blue'>B. Creating a Recognizer
    
    This generates a recognizer model, traininer.yml.
    
    Update the image_dir path.

In [29]:
# image file directory
image_dir = r'C:\Users\ML\Documents\faceID\image'

# face classifier
face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()

current_id = 0
label_ids = {}
x_train = [] # list to hold all subject faces
y_labels = [] # list to hold labels for all subjects

for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('jpg') or file.endswith('png'):
            path = os.path.join(root, file)
            label = os.path.basename(os.path.dirname(path)).replace(' ','-').lower()
#             print(label, path)
            
            # ID nubmer and label dictionary
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1
                
            id_ = label_ids[label]
#             print(label_ids)
            
            pil_image = Image.open(path).convert('L')  # open image and convert to grayscale
        
#             # scale image
#             size = (550, 550)
#             pil_image = pil_image.resize((600,600), Image.ANTIALIAS)
        
            image_array =  np.array(pil_image, 'uint8')
#             print(image_array)

            faces = face_cascade.detectMultiScale(image_array, scaleFactor = 1.5, minNeighbors = 5)
            
            for (x,y,w,h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)
            
# print(frame)
# print(x_train)

with open('labels.pickle', 'wb') as f:
    pickle.dump(label_ids, f)
    
recognizer.train(x_train, np.array(y_labels))
# recognizer.save('trainner.yml')
recognizer.write('trainner.yml')
print('Model Completed')

Model Completed


# <font color='blue'>C. Implement Recognizer
    
    Run this after you generated a recognizer mode, trainner.yml.

In [28]:
import numpy as np
import cv2
import pickle

# classifier input
face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trainner.yml')

# labeling name
labels = {}
with open('labels.pickle', 'rb') as f:
    old_labels = pickle.load(f)
    labels = {v:k for k, v in old_labels.items()}

# video w/rectangle
cap = cv2.VideoCapture(0)
color = (255, 0, 0) #BGR
stroke = 2
font = cv2.FONT_HERSHEY_SIMPLEX
colorFont = (255,255,255)



while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Display grayscale frames
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.5, minNeighbors=5)
    for (x, y, w, h) in faces:

        # region of interest
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # predict
        id_, conf = recognizer.predict(roi_gray)
#         print(conf)
        if conf >=45 and conf <= conf <= 100:
            print(id_)
            print(labels[id_])
            
            # label on the face rectangle
            
            name = labels[id_]
            cv2.putText(frame, name, (x,y), font, 1, colorFont, stroke, cv2.LINE_AA)

        img_item = 'my-image.png'
        cv2.imwrite(img_item, roi_gray)  # save roi image

        # rectable identifying faces
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x,y), (end_cord_x, end_cord_y), color, stroke)

    cv2.imshow('frame',frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

                

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
5
mooyoung-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
2
jesmi-lee
5
mooyoung-lee
2
jesmi-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
5
mooyoung-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
5
mooyoung-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2
jesmi-lee
2